In [101]:
import collections
import math

import numpy as np
import torch
from transformers import AutoTokenizer, AutoConfig, BertConfig, BertTokenizer
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor, squad_convert_examples_to_features, SquadExample
from transformers import squad_convert_examples_to_features

In [69]:
qas_id = 'u_1'
title = 'bla'
question = 'Hội nghị Lập pháp ở Pháp đã tồn tại hai phe đối lập nào?'
answer = "nhóm 'Gironde' ủng hộ chiến tranh với Áo và Phổ, và nhóm 'Montagne' hoặc 'Jacobin' phản chiến"
context = "Trong tháng 8 năm 1791, Hoàng đế Áo và Quốc vương Phổ trong Tuyên ngôn Pillnitz đe doạ nước Pháp cách mạng về việc can thiệp vũ lực nhằm khôi phục chế độ quân chủ chuyên chế. Trong tháng 9 năm 1791, Quốc hội lập hiến buộc Quốc vương Louis XVI chấp thuận Hiến pháp Pháp 1791, theo đó biến Pháp từ quốc gia theo chế độ quân chủ chuyên chế thành chế độ quân chủ lập hiến. Trong Hội nghị Lập pháp mới thành lập (tháng 10 năm 1791), tình trạng thù định phát triển và sâu sắc giữa nhóm 'Gironde' ủng hộ chiến tranh với Áo và Phổ, và nhóm 'Montagne' hoặc 'Jacobin' phản chiến. Ngày 20 tháng 4 năm 1792, Hội nghị Lập pháp tuyên chiến với Áo."
s_pos = 475 
e_pos = 568
test = InputSquad(question, answer, context, s_pos, e_pos)

In [102]:
context

"Trong tháng 8 năm 1791, Hoàng đế Áo và Quốc vương Phổ trong Tuyên ngôn Pillnitz đe doạ nước Pháp cách mạng về việc can thiệp vũ lực nhằm khôi phục chế độ quân chủ chuyên chế. Trong tháng 9 năm 1791, Quốc hội lập hiến buộc Quốc vương Louis XVI chấp thuận Hiến pháp Pháp 1791, theo đó biến Pháp từ quốc gia theo chế độ quân chủ chuyên chế thành chế độ quân chủ lập hiến. Trong Hội nghị Lập pháp mới thành lập (tháng 10 năm 1791), tình trạng thù định phát triển và sâu sắc giữa nhóm 'Gironde' ủng hộ chiến tranh với Áo và Phổ, và nhóm 'Montagne' hoặc 'Jacobin' phản chiến. Ngày 20 tháng 4 năm 1792, Hội nghị Lập pháp tuyên chiến với Áo."

In [91]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
# tokenizer = AutoTokenizer.from_pretrained(MODEL, do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained('NlpHUST/vibert4news-base-cased', do_lower_case=False)

Loading BERT tokenizer...


In [100]:
# print(' Original: ', context)
# print('Tokenized: ', tokenizer.tokenize(context))
# print('Token IDs: ', tokenizer.convert_ids_to_tokens(tokenizer.encode(context)))

In [141]:
def train_enc(qas_ids, questions, titles, contexts, answers, is_impossibles, s_poses, tokenizer, args):
    datas = []
    for i in range(len(qas_ids)):
        data = SquadExample(qas_id=qas_ids[i], question_text=questions[i], context_text=contexts[i], title=titles[i], answer_text=answers[i], answers=[{'answer_start':s_poses[i],'text':answers[i]}], is_impossible=is_impossibles[i], start_position_character=s_poses[i])
        datas.append(data)
    
    features = squad_convert_examples_to_features(
        examples=[datas],
        tokenizer=tokenizer,
        max_seq_length= args.max_seq_length,
        doc_stride= args.doc_stride,
        max_query_length= args.max_query_length,
        is_training=True,
    )
    
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
#     all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
    all_start_pos =  torch.tensor([f.start_position for f in features], dtype=torch.long)
    all_end_pos = torch.tensor([f.end_position for f in features], dtype=torch.long)
    
    train_enc = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_start_pos, all_end_pos)
    return train_enc

In [162]:
def valid_enc(qas_ids, questions, titles, contexts, answers, is_impossibles, s_poses, tokenizer, args):
    datas = []
    for i in range(len(qas_ids)):
        data = SquadExample(qas_id=qas_ids[i], question_text=questions[i], context_text=contexts[i], title=titles[i], answer_text=answers[i], answers=[{'answer_start':s_poses[i],'text':answers[i]}], is_impossible=is_impossibles[i], start_position_character=s_poses[i])
        datas.append(data)
    
    features = squad_convert_examples_to_features(
        examples=[datas],
        tokenizer=tokenizer,
        max_seq_length= args.max_seq_length,
        doc_stride= args.doc_stride,
        max_query_length= args.max_query_length,
        is_training=True,
    )
    
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_example_index = torch.arange([f.example_index for f in features], dtype=torch.long)
    
    valid_enc = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
    return datas, valid_enc

In [ ]:
def test_enc(qas_ids, questions, titles, contexts, tokenizer, args):
    datas = []
    for i in range(len(qas_ids)):
        data = SquadExample(qas_id=qas_ids[i], question_text=questions[i], context_text=contexts[i], title=titles[i], answer_text="", answers=[{'answer_start':-1,'text':""}], is_impossible=False, start_position_character=-1)
        datas.append(data)
    
    features = squad_convert_examples_to_features(
        examples=[datas],
        tokenizer=tokenizer,
        max_seq_length= args.max_seq_length,
        doc_stride= args.doc_stride,
        max_query_length= args.max_query_length,
        is_training=True,
    )
    
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_example_index = torch.arange([f.example_index for f in features], dtype=torch.long)
    
    valid_enc = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
    return datas, valid_enc

In [156]:
example = [SquadExample(qas_id=qas_id, question_text=question, context_text=context, title=title, answer_text=answer, answers=[{'answer_start':s_pos,'text':answer}],is_impossible=False, start_position_character=s_pos)]

In [157]:
features = squad_convert_examples_to_features(
    examples=example,
    tokenizer=tokenizer,
    max_seq_length=100,
    doc_stride=50,
    max_query_length=128,
    is_training=True,
)


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 34952.53it/s]


In [158]:
features[0].token_to_orig_map

{17: 0,
 18: 1,
 19: 2,
 20: 3,
 21: 4,
 22: 4,
 23: 5,
 24: 6,
 25: 7,
 26: 8,
 27: 9,
 28: 10,
 29: 11,
 30: 12,
 31: 13,
 32: 14,
 33: 15,
 34: 15,
 35: 15,
 36: 16,
 37: 17,
 38: 18,
 39: 19,
 40: 20,
 41: 21,
 42: 22,
 43: 23,
 44: 24,
 45: 25,
 46: 26,
 47: 27,
 48: 28,
 49: 29,
 50: 30,
 51: 31,
 52: 32,
 53: 33,
 54: 34,
 55: 35,
 56: 36,
 57: 36,
 58: 37,
 59: 38,
 60: 39,
 61: 40,
 62: 41,
 63: 41,
 64: 42,
 65: 43,
 66: 44,
 67: 45,
 68: 46,
 69: 47,
 70: 48,
 71: 49,
 72: 50,
 73: 51,
 74: 52,
 75: 53,
 76: 54,
 77: 55,
 78: 56,
 79: 56,
 80: 57,
 81: 58,
 82: 59,
 83: 60,
 84: 61,
 85: 62,
 86: 63,
 87: 64,
 88: 65,
 89: 66,
 90: 67,
 91: 68,
 92: 69,
 93: 70,
 94: 71,
 95: 72,
 96: 73,
 97: 74,
 98: 75}

In [160]:
example[features[0].example_index].doc_tokens

['Trong',
 'tháng',
 '8',
 'năm',
 '1791,',
 'Hoàng',
 'đế',
 'Áo',
 'và',
 'Quốc',
 'vương',
 'Phổ',
 'trong',
 'Tuyên',
 'ngôn',
 'Pillnitz',
 'đe',
 'doạ',
 'nước',
 'Pháp',
 'cách',
 'mạng',
 'về',
 'việc',
 'can',
 'thiệp',
 'vũ',
 'lực',
 'nhằm',
 'khôi',
 'phục',
 'chế',
 'độ',
 'quân',
 'chủ',
 'chuyên',
 'chế.',
 'Trong',
 'tháng',
 '9',
 'năm',
 '1791,',
 'Quốc',
 'hội',
 'lập',
 'hiến',
 'buộc',
 'Quốc',
 'vương',
 'Louis',
 'XVI',
 'chấp',
 'thuận',
 'Hiến',
 'pháp',
 'Pháp',
 '1791,',
 'theo',
 'đó',
 'biến',
 'Pháp',
 'từ',
 'quốc',
 'gia',
 'theo',
 'chế',
 'độ',
 'quân',
 'chủ',
 'chuyên',
 'chế',
 'thành',
 'chế',
 'độ',
 'quân',
 'chủ',
 'lập',
 'hiến.',
 'Trong',
 'Hội',
 'nghị',
 'Lập',
 'pháp',
 'mới',
 'thành',
 'lập',
 '(tháng',
 '10',
 'năm',
 '1791),',
 'tình',
 'trạng',
 'thù',
 'định',
 'phát',
 'triển',
 'và',
 'sâu',
 'sắc',
 'giữa',
 'nhóm',
 "'Gironde'",
 'ủng',
 'hộ',
 'chiến',
 'tranh',
 'với',
 'Áo',
 'và',
 'Phổ,',
 'và',
 'nhóm',
 "'Montagne'",
 'hoặ